## RSI - ML prediction based long time-frame strategy ( 1 Day)
* Using 3 technical indicators - RSI(9), EMA(3), WMA(21) 

#### Importing libraries

In [4]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

import warnings
# warnings.warn('Error: A warning just appeared')
warnings.filterwarnings('ignore')

### Data Preprocessing
* Checking for Nan values
* Replacing Nan with the average of previous and next value

In [1]:
import pandas as pd

def fill_nan_with_avg(df):
    # Forward fill NaN values
    df_ffill = df.ffill()

    # Backward fill NaN values
    df_bfill = df.bfill()

    # Calculate the average of preceding and succeeding values
    avg_values = (df_ffill + df_bfill) / 2

    # Replace NaN values in the original DataFrame with the calculated averages
    df_filled = df.where(df.notna(), avg_values)

    return df_filled

### Arima Prediction
* Prediciting the next candle for the last row using `ARIMA` model

#### Data Processing

In [ ]:
def data_process(df):
    df_for_training = df[:]
    history = [i for i in df_for_training["close"]]
    return history

#### Predictions using ARIMA

* Predicting `close` price using ARIMA

In [ ]:
def predict(history):
    predictions = []
    for t in range(1):
        model = ARIMA(history, order=(5,1,0))
        model_fit = model.fit()
        output = model_fit.forecast()
        # print(output)
        yhat = output[0]
        predictions.append(yhat)
    return predictions

#### Appending the predictions ("close") to the dataframe

In [ ]:
def add(csv_path, val):
    df = pd.read_csv(csv_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    time_diff = df['datetime'].diff().iloc[-1]
    next_datetime = df['datetime'].iloc[-1] + time_diff
    data = {'datetime': [next_datetime],
        'close': [val]}
    new_row = pd.DataFrame(data)
    df = pd.concat([df, new_row], ignore_index=True)
    df = df.sort_values(by='datetime').reset_index(drop=True)
    print("New DataFrame created Successfully....")
    return df

## Strategy

In [ ]:
class TradingStrategy:
    def __init__(self, data):
        self.data = data  # represent the dataset
        self.inMarket = 0 # if it is 0 then there is no trade and if it is 1 then there is a trade going on
        self.buy = 0 # if it is 1 then we bought a trade in long side and if it is 2 then we sell a trade for short position and 0 for nothing

    def generate_signals(self):  # function for generate signals in long and short position
        for i in range(1, len(self.data) - 1): # loop through the data
            if self.inMarket == 0: # if there is no trade going on then check condition for long or short position
                if (
                    self.data['EMA3'][i - 1] < self.data['WMA'][i - 1] and
                    self.data['RSI'][i + 1] > self.data['EMA3'][i + 1] and
                    self.data['EMA3'][i + 1] > self.data['WMA'][i + 1]
                ):
                    self.data['Signal'][i] = 1  # put signal 1 for buying a trade in long side 
                    self.buy = 1 
                    self.inMarket = 1

                if (
                    self.data['EMA3'][i - 1] > self.data['WMA'][i - 1] and
                    self.data['RSI'][i + 1] < self.data['EMA3'][i + 1] and
                    self.data['EMA3'][i + 1] < self.data['WMA'][i + 1]
                ):
                    self.data['Signal'][i] = -1 # put signal -1 for sell a trade for short position
                    self.buy = 2  
                    self.inMarket = 1

            elif self.inMarket == 1: # if there is trade  going on and check if buy value is 1 then we will sell the trade of long position
                if self.buy == 1 and self.data['RSI'][i - 1] > self.data['EMA3'][i - 1] and self.data['RSI'][i + 1] < self.data['EMA3'][i + 1]:
                    self.data['Signal'][i] = -1  # signal -1 for selling the trade of long position
                    self.inMarket = 0
                    self.buy = 0
                # if buy value is 2 then we will buy a trade for short position 
                elif self.buy == 2 and self.data['RSI'][i - 1] < self.data['EMA3'][i - 1] and self.data['RSI'][i + 1] > self.data['EMA3'][i + 1]:
                    self.data['Signal'][i] = 1
                    self.buy = 0
                    self.inMarket = 0

        # if sum = 0 and signal = 1 then long entry
        # if sum = 0 and signal = -1 then short entry
        # if sum != 0 and signal = -1 then long exit 
        # if sum != 0 and signal = 1 then short exit
        for i in range(len(self.data)): 
            self.data['sum'][i] = self.data['sum'][i - 1] + self.data['Signal'][i - 1]

class TradingSystem:
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path) # reading the csv data file
        self.data.rename(columns = {'datetime':'timestamp'}, inplace = True) # rename the datetime column as timestamp
        self.data['timestamp'] = pd.to_datetime(self.data['timestamp'])  # set the datetime in correct format
        self.data.set_index('timestamp', inplace=True)
        self.data['RSI'] = ta.RSI(self.data['close'], timeperiod=9) # using RSI indicator on close and length is 9
        self.data['EMA3'] = ta.EMA(self.data['RSI'], timeperiod=3)  # using EMA indicator on RSI and length is 3
        self.data['WMA'] = ta.WMA(self.data['RSI'], timeperiod=21)  # using WMA indicator on RSI and length is 21
        self.data['Signal'] = 0  # using signal column for checking entry or exit in long and short position
        self.data['sum'] = 0 # using sum column for checking the current trade in long or short position

        self.strategy = TradingStrategy(self.data) 

    def run_strategy(self):
        self.strategy.generate_signals() 
        self.data.to_csv("Result.csv")  # save the signal data in csv file
#         print("Total number of trades: ", self.strategy.countLongTrades + self.strategy.countShortTrades)

# if __name__ == "__main__":
#     warnings.filterwarnings("ignore") # ignore the warnings 
#     trading_system = TradingSystem("DailyData.csv") # giving the dataset 
#     trading_system.run_strategy() # call the function

    
    
####----> BACKTEST RESULTS <----####
 
# Gross Profit --> 10350.54213584676
# Net Profit  --> 10109.664469421492
# Positive PnL --> 10554.164825488748
# negative PnL --> -203.62268964198896
# Total number of closed trades --> 163
# Win Rate(%) --> 89.57055214723927
# Max Drawdown(%) --> -3.2318821997125164
# Gross Loss --> 0
# Average Winning Trade (USDT) --> 72.28880017458046
# Average Losing Trade (USDT) --> -11.977805273058173
# Buy and Hold Return of BTC(%) --> 228.121525
# Largest Losing Trade (USDT) --> -32.58005517839974
# Largest Winning Trade (USDT) --> 459.7788469654765
# Sharpe Ratio --> 12.653810490851289
# Sortino Ratio --> 201.63303615421327
# Average Holding Duration per Trade --> 11 Days
# Equity Final [$]  ----> 12933542.116158
# Equity Peak [$]   ----> 12946475.658264
# Return [%]       ---->  1293254.211616
# Buy & Hold Return [%]   ----> 228.121525
# Return (Ann.) [%]    ----> 944.37713

#### Reading the dataframe

In [ ]:
data = pd.read_csv("btcusdt_15m.csv")

#### Replacing Nan values with the Average of preceding and succeeding

In [ ]:
data = fill_nan_with_avg(data)

#### Prediction using ARIMA 

In [ ]:
new_df = add("btcusdt_15m.csv",predict(data_process(data))[0])

#### Saving the dataframe

In [ ]:
new_df.to_csv("btcusdt_15m_new.csv")

### Using the new dataframe to get the Signal

* Saving the result as `Result.csv`

In [ ]:
trading_system = TradingSystem("btcusdt_15m_new.csv") # giving the dataset 
trading_system.run_strategy()